# Shortest path algorithms
Shortest path is the core of operations research.
The shortest path algorithms can be devided into many categories, such as:
1. Single objective VS multiobjective
2. Deterministic VS stochastic
3. Addtive VS nonadditive
4. Nonconstrained VS constrained
5. Static VS time-dependent\
...

I will try my best to cover all the categories and include some interesting variants.
The structure of this document will be:
1. Mathematical model.
2. Data structure.
3. Algorihtm.
4. Data.
5. Implementation.

## Bi-objective, deterministic, additive shortest path

#### From textbook: Ehrgott, M. (2005). Multicriteria optimization (Vol. 491). Springer Science & Business Media.
<img src='images/Martin_algorithm.jpg' width='700'>

In [214]:
# Data Stucture
class Node:
    """
    This class has attributes associated with any node
    """
    def __init__(self, Id):
        self.Id = Id
        self.outLinks = []
        self.inLinks = []
        '''Algorithm'''
        self.labels = {} # labels: dictionay

class Link:
    """
    This class has attributes associated with any bi-attribute link
    """
    def __init__(self, tail, head, cost1, cost2):
        self.tail = tail
        self.head = head
        self.cost1 = cost1
        self.cost2 = cost2
        
class Bi_Obj_Label():
    '''
    This class is defined for recording bi-objective labels.
    '''
    def __init__(self, Id, node, cost1, cost2, pred_node, pred_label):
        self.Id = Id
        self.node = int(node)
        self.cost1 = float(cost1)
        self.cost2 = float(cost2)
        self.pred_node = pred_node
        self.pred_label = pred_label

In [215]:
def Bi_Obj_LS(nodeSet, linkSet, ori, des=None):

    # initialization
    label_Id = 0
    for n in nodeSet:
        nodeSet[n].labels[label_Id] = Bi_Obj_Label(label_Id, n, float("inf"), float("inf"), 'NA', 'NA')
        label_Id += 1
    # set origin
    for b in nodeSet[ori].labels:
        nodeSet[ori].labels[b].cost1, nodeSet[ori].labels[b].cost2 = (0, 0)
        SE = [(0, 0, b, ori)]
    
    while SE:
        # lexicographical sort?
        this_cost1, this_cost2, this_label, this_node = SE.pop(0)
        
        for l in nodeSet[this_node].outLinks:
            next_node = l[1]
            new_cost1, new_cost2 = (this_cost1 + linkSet[l].cost1, this_cost2 + linkSet[l].cost2)
            
            # check dominance
            nondominance = True
            for b in list(nodeSet[next_node].labels.keys()):
                exsiting_cost1, exsiting_cost2 = (nodeSet[next_node].labels[b].cost1, nodeSet[next_node].labels[b].cost2)
                
                if exsiting_cost1 <= new_cost1 and exsiting_cost2 <= new_cost2: # new label is dominated by old label
                    nondominance = False
                    break
                elif new_cost1 <= exsiting_cost1 and new_cost2 <= exsiting_cost2: # old label is dominanted by new label
                    nodeSet[next_node].labels.pop(b)
            
            if nondominance == True:
                # create a new label at next_node and add it to the SE list
                nodeSet[next_node].labels[label_Id] = Bi_Obj_Label(label_Id, next_node, new_cost1, new_cost2, this_node, this_label)
                SE.append((new_cost1, new_cost2, label_Id, next_node))
                label_Id += 1
    
    # construct efficient paths and distances
    if des != None:
        path, dist = ([], [])
        for b in nodeSet[des].labels:
            path.append(trace_efficient_path(des, b))
            dist.append((nodeSet[des].labels[b].cost1, nodeSet[des].labels[b].cost2))
    else:
        path, dist = ({}, {})
        for n in nodeSet:
            tmp_path, tmp_dist = ([], [])
            for b in nodeSet[n].labels:
                tmp_path.append(trace_efficient_path(n, b))
                tmp_dist.append((nodeSet[n].labels[b].cost1, nodeSet[n].labels[b].cost2))                
            path[n], dist[n] = (tmp_path, tmp_dist)
    
    return path, dist

def trace_efficient_path(des, label):
    """
    This method traverses predecessor nodes in order to create a efficient path.
    """
    this_node, this_label = (des, label)
    pathNodes = [this_node]
    
    while nodeSet[this_node].labels[this_label].pred_node != 'NA':
        pathNodes.append(nodeSet[this_node].labels[this_label].pred_node)
        this_node, this_label = (nodeSet[this_node].labels[this_label].pred_node,
                                 nodeSet[this_node].labels[this_label].pred_label)
    list.reverse(pathNodes)
    
    return pathNodes

### Data

#### From textbook: Ehrgott, M. (2005). Multicriteria optimization (Vol. 491). Springer Science & Business Media.
<img src="images/Textbook_network.jpg" width="800">

Use the first two numbers at each link as the link cost.

In [216]:
'Data'
# tail, head, cost
linkData = [
    [1, 2, 10, 4],
    [1, 3, 6, 1],
    [2, 4, 0, 10],
    [3, 2, 6, 2],
    [3, 5, 1, 4],
    [4, 3, 4, 0],
    [4, 6, 10, 1],
    [5, 4, 5, 1],
    [5, 6, 6, 0],
]

In [217]:
'Read data'
nodeSet = {}
linkSet = {}

for l in linkData:
    tail, head, cost1, cost2 = l
    # link
    linkSet[tail, head] = Link(tail, head, cost1, cost2)
    # node
    if tail not in nodeSet:
        nodeSet[tail] = Node(tail)
    if head not in nodeSet:
        nodeSet[head] = Node(head)
    if (tail, head) not in nodeSet[tail].outLinks:
        nodeSet[tail].outLinks.append((tail, head))
    if (tail, head) not in nodeSet[head].inLinks:
        nodeSet[head].inLinks.append((tail, head))

print(len(nodeSet), "nodes")
print(len(linkSet), "links")

'Implementation'
Ori = 1
Des = 4
path, dist = Bi_Obj_LS(nodeSet, linkSet, Ori, Des)
print('Shortest path', path)
print('Cost', dist)

6 nodes
9 links
Shortest path [[1, 2, 4], [1, 3, 5, 4]]
Cost [(10.0, 14.0), (12.0, 6.0)]


### What about the performance in big networks?
#### Chicago Sketch network: 933 nodes, 2950 links
<img src="images/Chicago_Sketch.png" width="600">

In [218]:
# Chicago Sketch network

import time

netFile = 'network\\ChicagoSketch_Bi-objective.csv'

nodeSet = {}
linkSet = {}

inFile = open(netFile, 'r')
next(inFile)  # skip the first title line
for line in inFile:
    # data
    tmpIn = line.strip('\n').split(',')
    tail = int(tmpIn[0])
    head = int(tmpIn[1])
    cost1 = float(tmpIn[2])
    cost2 = float(tmpIn[3])
    # link
    linkSet[tail, head] = Link(tail, head, cost1, cost2)
    # node
    if tail not in nodeSet:
        nodeSet[tail] = Node(tail)
    if head not in nodeSet:
        nodeSet[head] = Node(head)
    if (tail, head) not in nodeSet[tail].outLinks:
        nodeSet[tail].outLinks.append((tail, head))
    if (tail, head) not in nodeSet[head].inLinks:
        nodeSet[head].inLinks.append((tail, head))

print(len(nodeSet), "nodes")
print(len(linkSet), "links")

'Implementation'
Ori = 1
Des = 400

tic = time.time()
path, dist = Bi_Obj_LS(nodeSet, linkSet, Ori, Des)
print('Running time=', time.time()-tic, 'sec.')

print('Shortest path', path)
print('Cost', dist)

933 nodes
2950 links
Running time= 0.36620330810546875 sec.
Shortest path [[1, 547, 548, 552, 435, 436, 437, 438, 536, 537, 399, 398, 400], [1, 547, 548, 552, 435, 554, 437, 438, 536, 537, 399, 398, 400], [1, 547, 621, 620, 598, 599, 432, 431, 593, 594, 589, 590, 401, 400], [1, 547, 621, 620, 616, 433, 432, 431, 593, 594, 589, 590, 401, 400], [1, 547, 548, 552, 435, 436, 437, 438, 536, 537, 608, 609, 399, 398, 400], [1, 547, 548, 552, 435, 554, 437, 438, 536, 537, 608, 609, 399, 398, 400], [1, 547, 621, 620, 598, 599, 432, 431, 428, 427, 594, 589, 590, 401, 400], [1, 547, 621, 620, 616, 433, 432, 431, 428, 427, 594, 589, 590, 401, 400]]
Cost [(30.23, 34.433152), (29.779999999999998, 35.389492), (38.92, 26.382848), (39.2, 25.430504), (34.56, 32.595156), (34.11, 33.551496), (39.08, 25.938912), (39.36, 24.986568)]
